In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

Preparing the data

In [60]:
dataset = pd.read_csv("/content/drive/MyDrive/Machine Learning/train.csv")
dataset = dataset.dropna()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
heroes = dataset[['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero', 'radiant_win']]
heroes.head()

In [62]:
labels = dataset[['radiant_win']]

Split the data into testing and training

In [63]:
data_train = heroes.sample(frac=0.8, random_state=0)
data_test = heroes.drop(data_train.index)

Since all the data is categorical, it needs to be vectorices into a binary vector

In [64]:
def vectorice_sequences(sequences):
    results = np.zeros((len(sequences),113))

    for i,sequence in enumerate(sequences):
        results[i, sequence] = 1

    return results

def convert_data(df):
    #creates a vector which has 226 values. The coordinates are a index is determined by the hero and team choice.
    radiant = dataset[['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero']]
    dire = dataset[['d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']]
 
    dire = tf.convert_to_tensor(dire)
    radiant = tf.convert_to_tensor(radiant)

    radiant = vectorice_sequences(radiant)
    dire = vectorice_sequences(dire)

    data = tf.concat([dire, radiant], -1)
    
    return data


data_train = convert_data(data_train)
data_test = convert_data(data_test)

Split the labels from the training data

In [65]:
labels_train = labels.sample(frac=0.8, random_state=0)
labels_test = labels.drop(labels_train.index)

In [66]:
def convert_labels(labels):

    labels.loc[labels['radiant_win'] == True, 'radiant_win'] = 1
    labels.loc[labels['radiant_win'] == False, 'radiant_win'] = 0

    labels = np.squeeze(labels)

    return labels 

#turn booleans to binary values
labels_train = convert_labels(labels_train)
labels_test = convert_labels(labels_test)

labels_train = np.asarray(labels_train).astype('float32')
labels_test = np.asarray(labels_test).astype('float32')

Building, trainign and evaluating the model

In [ ]:
from keras.models import Sequential
from keras import models
from keras import layers

#build the model
model = Sequential()
model.add(layers.Dense(16,activation = 'relu', input_shape = (226,)))
model.add(layers.Dense(32,activation = 'relu', input_shape = (226,)))

model.add(layers.Dropout(.2))

model.add(layers.Dense(64,activation = 'relu', input_shape = (226,)))
model.add(layers.Dense(128,activation = 'relu', input_shape = (226,)))

model.add(layers.Dense(1, activation= 'sigmoid'))

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

model.summary()

Train the model...

In [57]:
history = model.fit(data_train,labels_train,epochs = 100, validation_split=0.2, verbose = 0 )

And evaluate it.

In [58]:
results = model.evaluate(data_test,labels_test)

467/467 [==============================] - 1s 1ms/step - loss: 0.7514 - accuracy: 0.5604


In [45]:
model.save("/content/drive/MyDrive/Machine Learning/Dota_model.h5")